In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from functools import reduce
import matplotlib as plt
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [8]:
#import csv of oscar winners
cast_awards_2016=pd.read_csv("../DataSources/Oscars2016.csv")

cast_awards_2016

,Title,Best Director,Best Actor,Best Actress,Best Supporting Actor,Best Supporting Actress
0,La La Land,1,0,1,0,0
1,Arrival,0,0,0,0,0
2,Lion,0,0,0,0,0
3,Hell or High Water,0,0,0,0,0
4,Hidden Figures,0,0,0,0,0
5,Moonlight,0,0,0,1,0
6,Hacksaw Ridge,0,0,0,0,0
7,Manchester by the Sea,0,1,0,0,0
8,Fences,0,0,0,0,1


In [9]:
#OMBD Calls - done on movies list
theMoviesDF_OMBD = pd.read_csv('../DataSources/theMoviesDF2016_OMBD.csv')
theMoviesDF_OMBD.head()

,Actors,Awards,Box Office,Country,Director,Genre,Rated,Title,Writer(s),Year,imdbRating
0,"Ryan Gosling, Emma Stone, Amiée Conn, Terry Wa...",Won 6 Oscars. Another 215 wins & 254 nominations.,"$151,058,124","USA, Hong Kong",Damien Chazelle,"Comedy, Drama, Music, Musical, Romance",PG-13,La La Land,Damien Chazelle,2016,8.0
1,"Amy Adams, Jeremy Renner, Forest Whitaker, Mic...",Won 1 Oscar. Another 64 wins & 251 nominations.,"$100,501,349",USA,Denis Villeneuve,"Drama, Mystery, Sci-Fi, Thriller",PG-13,Arrival,"Eric Heisserer (screenplay by), Ted Chiang (ba...",2016,7.9
2,"Sunny Pawar, Abhishek Bharate, Priyanka Bose, ...",Nominated for 6 Oscars. Another 49 wins & 80 n...,"$51,694,854","UK, Australia, USA",Garth Davis,"Biography, Drama",PG-13,Lion,"Saroo Brierley (adapted from the book ""A Long ...",2016,8.1
3,"Dale Dickey, Ben Foster, Chris Pine, William S...",Nominated for 4 Oscars. Another 42 wins & 157 ...,"$26,973,524",USA,David Mackenzie,"Action, Crime, Drama, Thriller, Western",R,Hell or High Water,Taylor Sheridan,2016,7.6
4,"Taraji P. Henson, Octavia Spencer, Janelle Mon...",Nominated for 3 Oscars. Another 36 wins & 81 n...,"$169,385,416",USA,Theodore Melfi,"Biography, Drama, History",PG,Hidden Figures,"Allison Schroeder (screenplay by), Theodore Me...",2016,7.8


In [10]:
#TMDB Calls
theMoviesDF_TMDB=pd.read_csv('../DataSources/theMoviesDF2016_TMDB.csv')
theMoviesDF_TMDB.head()

,Title,Budget,Popularity,Release Date,Revenue,Runtime,Vote_average,Vote_count
0,La La Land,30000000,17.561,2016-11-29,445435700,129,7.9,9285
1,Arrival,47000000,20.369,2016-11-10,203388186,116,7.4,9707
2,Lion,12000000,24.197,2016-11-24,140302754,118,8.1,3479
3,Hell or High Water,12000000,12.255,2016-08-12,37589296,102,7.3,2289
4,Hidden Figures,25000000,19.442,2016-12-10,230698791,127,8.0,4360


In [11]:
#fix any title discrepancies between OMBD and TMDB dataframes by replacing incorrect OMBD names with TMDB names
#OMBD does not have data on the following movies: Precious: Based on the Novel 'Push' by Sapphire, The Postman (Il Postino), Cries and Whispers, Nashville 
bad_names_OMBD=[]
correct_names_TMDB=[]

for i in theMoviesDF_OMBD["Title"].values:
    if i in theMoviesDF_TMDB["Title"].values:
        pass
    else:
        bad_names_OMBD.append(i)


for i in theMoviesDF_TMDB["Title"].values:
    if i in theMoviesDF_OMBD["Title"].values:
        pass
    else:
        correct_names_TMDB.append(i)

bad_names_OMBD

[]

In [12]:
correct_names_TMDB

[]

In [14]:
theMoviesDF_OMBD.shape

(9, 11)

In [15]:
theMoviesDF_TMDB.shape

(9, 8)

# EXTRACTING INFO FROM API CALLS

In [16]:
#DataFrame by Genre (OMBD)
genreMovies=theMoviesDF_OMBD[['Title', 'Genre']]

for i in range(len(genreMovies['Genre'])):
    x=genreMovies['Genre'][i].split(", ")
    genreMovies['Genre'][i]=x

s=genreMovies.apply(lambda x:pd.Series(x['Genre']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'Genre'
BP_genres=genreMovies.drop('Genre', axis=1).join(s)
BP_genres=pd.get_dummies(BP_genres, columns=['Genre'])
BP_genres=BP_genres.groupby('Title').sum()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
#Dataframe by IMBD rating (OMBD)
BP_IMBD=theMoviesDF_OMBD[['Title','imdbRating']]

#Dataframe by Rating (OMBD)
BP_Rating_qual=theMoviesDF_OMBD[['Title','Rated']]
#consolidate certain ratings
BP_Rating_qual['Rated']=BP_Rating_qual['Rated'].replace({'GP':'PG',
                                                        'M/PG': 'PG',
                                                        'TV-MA': 'R',
                                                        'TV-PG': 'PG',
                                                        'PASSED':'APPROVED',
                                                        'NOT RATED': 'UNRATED'})
BP_Rating=pd.get_dummies(BP_Rating_qual, columns=['Rated'])

#Dataframe by Country (OMBD)
BP_Country_qual=theMoviesDF_OMBD[['Title','Country']]
BP_Country=pd.get_dummies(BP_Country_qual, columns=['Country'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [18]:
#Dataframe by Revenue and Budget (TMDB)
BP_Revenue_Budget=theMoviesDF_TMDB[['Title','Revenue', 'Budget']]

#Dataframe by Popularity (TMDB)
BP_Popularity = theMoviesDF_TMDB[['Title','Popularity']]

#Dataframe by Run Time (TMDB)
BP_Runtime = theMoviesDF_TMDB[['Title','Runtime']]

In [19]:
#Dataframe for Release Month (TMDB)
BP_RelMonth_qual = theMoviesDF_TMDB[['Title','Release Date']]
for i in range(len(BP_RelMonth_qual['Release Date'])):
    x=BP_RelMonth_qual['Release Date'][i].split("-")
    BP_RelMonth_qual['Release Date'][i]=x[1]

BP_RelMonth=pd.get_dummies(BP_RelMonth_qual, columns=['Release Date'])

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2961: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
#create a spreadsheet of all info (cannot have string as values)
theMoviesDF=pd.merge(cast_awards_2016, BP_IMBD, on= 'Title', how='inner')
dfs=[theMoviesDF, BP_Popularity, BP_Rating, BP_RelMonth, BP_Revenue_Budget, BP_Runtime, BP_genres]
theMoviesDF= reduce(lambda df1,df2: pd.merge(df1,df2,on=['Title'], how='left'), dfs)
theMoviesDF.fillna(0, inplace=True)
theMoviesDF.to_csv('../DataSources/theMovieFeatures2016.csv', index=False)
theMoviesDF

,Title,Best Director,Best Actor,Best Actress,Best Supporting Actor,Best Supporting Actress,imdbRating,Popularity,Rated_PG,Rated_PG-13,...,Genre_Drama,Genre_History,Genre_Music,Genre_Musical,Genre_Mystery,Genre_Romance,Genre_Sci-Fi,Genre_Thriller,Genre_War,Genre_Western
0,La La Land,1,0,1,0,0,8.0,17.561,0,1,...,1,0,1,1,0,1,0,0,0,0
1,Arrival,0,0,0,0,0,7.9,20.369,0,1,...,1,0,0,0,1,0,1,1,0,0
2,Lion,0,0,0,0,0,8.1,24.197,0,1,...,1,0,0,0,0,0,0,0,0,0
3,Hell or High Water,0,0,0,0,0,7.6,12.255,0,0,...,1,0,0,0,0,0,0,1,0,1
4,Hidden Figures,0,0,0,0,0,7.8,19.442,1,0,...,1,1,0,0,0,0,0,0,0,0
5,Moonlight,0,0,0,1,0,7.4,11.615,0,0,...,1,0,0,0,0,0,0,0,0,0
6,Hacksaw Ridge,0,0,0,0,0,8.1,19.047,0,0,...,1,1,0,0,0,0,0,0,1,0
7,Manchester by the Sea,0,1,0,0,0,7.8,10.412,0,0,...,1,0,0,0,0,0,0,0,0,0
8,Fences,0,0,0,0,1,7.2,10.158,0,1,...,1,0,0,0,0,0,0,0,0,0
